<a href="https://colab.research.google.com/github/AlexChek51/Book_Annotator/blob/main/Book_Annotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf tiktoken python-dotenv langchain langchain-openai langchain_community langchain-text-splitters 'arize-phoenix[evals]' nemoguardrails

In [ ]:
import nest_asyncio
import phoenix as px
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

# Настройка трассировки
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry import trace

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
trace.set_tracer_provider(tracer_provider)
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

tracer = trace.get_tracer(__name__)

# Запуск Phoenix после настройки трассировки
nest_asyncio.apply()
session = px.launch_app()

ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "llama-index-core >= 0.10.43" but found: "None"


🌍 To view the Phoenix app in your browser, visit https://g9flohhqei2-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
%%writefile .env
openai_api_key= введите ключ

Overwriting .env


In [ ]:
from dotenv import load_dotenv

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_openai import ChatOpenAI
from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import (
    StuffDocumentsChain,
    LLMChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
)
import os
from langchain.prompts import PromptTemplate

# Запросы
map_prompt = """Приведен текст части книги:
{text}
Тебе необходимо написать подробный пересказ объемов от 10% до 20% от исходного текста
Ответ должен быть на русском языке.
Ответ:
"""
reduce_prompt = """Перед тобой список пересказов частей текста книги:
{text_summaries}
Тебе необходимо объединить их в единый пересказ книги
Ответ должен быть на русском языке.
Ответ:
"""

prompt_full_book = """Напиши подробное изложение книги на русском языке.
Пожалуйста, убедись, что ответ будет на русском языке.
Объем изложения должен быть от 10% до 20% объема книги.
Проанализировать нужно всю книгу целиком.
Вот текст: {text}"""

user_question_prompt = """Вот суммаризированный текст книги:
{summary}

Теперь ответь на следующий вопрос:
{question}

Ответ должен быть на русском языке.
Ответ:
"""

MAP_PROMPT = PromptTemplate(input_variables=["text"], template=map_prompt)
REDUCE_PROMPT = PromptTemplate(input_variables=["text_summaries"], template=reduce_prompt)
PROMPT_FULL_BOOK = PromptTemplate.from_template(prompt_full_book)
USER_QUESTION_PROMPT = PromptTemplate(input_variables=["summary", "question"], template=user_question_prompt)

def run_mr(input_doc, MAP_PROMPT, REDUCE_PROMPT):
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_llm_chain,
        document_variable_name="text_summaries"
    )

    reduce_documents_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_documents_chain,
        collapse_documents_chain=combine_documents_chain,
        token_max=120000
    )

    combine_documents = MapReduceDocumentsChain(
        llm_chain=map_llm_chain,
        reduce_documents_chain=reduce_documents_chain,
        document_variable_name="text",
        return_intermediate_steps=False
    )

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=120000,
        chunk_overlap=0,
        separator="\n\n"
    )

    map_reduce = MapReduceChain(
        combine_documents_chain=combine_documents,
        text_splitter=text_splitter
    )

    return map_reduce.run(input_text=input_doc)

@tracer.start_as_current_span("full_book")
def full_book(book, prompt):
    with tracer.start_as_current_span("llm_chain") as span:
        llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
        span.set_attribute("model_name", llm.model_name)
        span.set_attribute("temperature", llm.temperature)
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        response = llm_chain.run(book)
        span.set_attribute("input_text", book)
        span.set_attribute("response", response)
    return response

In [ ]:
from dotenv import load_dotenv

# Загрузить переменные окружения из файла .env
load_dotenv()

import os
import tempfile
from langchain_community.document_loaders import PyPDFLoader
import tiktoken

def summarize_book(file_path: str, openai_api_key: str, user_question: str):
    with open(file_path, 'rb') as f:
        # Создаем временный файл и записываем в него содержимое файла
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(f.read())

        # Загружаем книгу из временного файла
        loader = PyPDFLoader(tmp_file.name)
        book = loader.load()

        # Удаляем временный файл
        os.remove(tmp_file.name)

    # Устанавливаем API ключ OpenAI
    os.environ["OPENAI_API_KEY"] = openai_api_key.strip()

    # Получаем текст книги
    text = "\n\n".join(page.page_content + "страница " + str(int(page.metadata['page']) + 1) for page in book)

    # Определяем метод суммаризации в зависимости от количества токенов
    encoding = tiktoken.encoding_for_model('gpt-4o')
    num_tokens = len(encoding.encode(text))
    with tracer.start_as_current_span("choose_summary_method") as span:
        span.set_attribute("num_tokens", num_tokens)
        if num_tokens > 120_000:
            summary = run_mr(text, MAP_PROMPT, REDUCE_PROMPT)
        else:
            summary = full_book(book, PROMPT_FULL_BOOK)

    # Формируем ответ на вопрос пользователя
    with tracer.start_as_current_span("user_question") as span:
        llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
        llm_chain = LLMChain(llm=llm, prompt=USER_QUESTION_PROMPT)
        response = llm_chain.run(summary=summary, question=user_question)
        span.set_attribute("user_question", user_question)
        span.set_attribute("response", response)

    return response

In [ ]:
openai_api_key = os.getenv("openai_api_key")
if not openai_api_key:
    print("API ключ OpenAI не настроен. Пожалуйста, настройте его и попробуйте снова.")
else:
    full_file_path = '/content/drive/MyDrive/split_pdf_chunks/chunk_10.pdf'  # Формируем полный путь к файлу
    user_question = "Каковы основные темы книги?"  # Пример вопроса пользователя
    response = summarize_book(full_file_path, openai_api_key, user_question)

    # Выводим результат суммаризации и ответа на вопрос
    print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
WARNI [opentelemetry.attributes] Invalid type Document in attribute 'input_text' value sequence. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or None


Основные темы книги "Война и мир" Льва Толстого включают:

1. **Война и мир**: Центральная тема романа, отражающая контраст между разрушительными последствиями войны и стремлением к мирной жизни. Толстой исследует, как война влияет на общество, семьи и индивидуальные судьбы.

2. **Судьба и свободная воля**: Толстой рассматривает вопрос о том, насколько люди контролируют свою судьбу и как внешние обстоятельства и исторические события влияют на их жизнь.

3. **Любовь и семейные отношения**: Взаимоотношения между членами семьи, романтические связи и дружба играют важную роль в романе. Толстой показывает, как любовь и поддержка близких помогают героям преодолевать трудности.

4. **Мораль и нравственность**: Герои романа часто сталкиваются с моральными дилеммами и вынуждены делать выбор между правильным и неправильным. Толстой исследует, как личные убеждения и общественные нормы влияют на поведение людей.

5. **Поиск смысла жизни**: Многие персонажи, особенно Пьер Безухов, стремятся найти с

1. Текст, подаваемый на вход, должен быть четко структурирован и соответствовать требованиям промпта. Наличие конкретных инструкций повышает точность генерации.
2. При использовании метода MapReduce важно следить за качеством частичных пересказов. Они должны полноценно отражать содержание оригинальных частей текста.
3. Итоговый пересказ должен объединять ключевые идеи всех частичных пересказов без потери важной информации. Важно избегать дублирования и обеспечить логическую связность текста.
4. Ответы на вопросы должны быть основаны на суммаризированном тексте и предоставлять точную информацию. Использование качественных промптов и корректных данных на входе способствует генерации адекватных ответов.

Анализ данных и ответов RAG системы позволяет выявить ключевые области для улучшения, такие как качество входных данных, структура промптов и точность промежуточных результатов, что в конечном итоге улучшает качество итогового пересказа и ответов на вопросы.

In [ ]:
colang_content = """
define user ask with profanity
    "Какие основнoe ерунда в этой книге?"
    "Что за хрень написана в этой книге?"
    "Какая чушь в этой книге?"

define user ask too long
    "Можете ли вы дать подробный анализ всех тем и подтем, рассматриваемых в этой книге, включая все возможные аспекты и детали, чтобы я мог получить полное представление о содержании и понять основные моменты и выводы?"

define user ask sensitive info
    "Что говорится в книге о [чувствительная информация]?"
    "Какие сведения в книге о [конфиденциальная информация]?"
    "Что автор пишет о [личная информация]?"

define bot handle profanity
    "Ваш запрос содержит нецензурные слова. Пожалуйста, переформулируйте свой запрос."

define bot handle too long
    "Ваш запрос слишком длинный. Он был усечен до допустимого объема."

define bot handle sensitive info
    "Ваш запрос содержит чувствительную информацию. Пожалуйста, переформулируйте свой запрос."

define flow handle profanity
    user ask with profanity
    bot handle profanity

define flow handle too long
    user ask too long
    bot handle too long

define flow handle sensitive info
    user ask sensitive info
    bot handle sensitive info
"""

yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-4o
"""


In [ ]:
from nemoguardrails import RailsConfig, LLMRails

config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)

rails = LLMRails(config)

async def get_response(messages):
    response = await rails.generate_async(messages=messages)
    return response

# Примеры запросов и ответы бота

# Пример 1: Запрос с нецензурной лексикой
messages = [
    {"role": "user", "content": "Какие основнoe ерунда в этой книге?"}
]
response = await get_response(messages)
print(response)

# Пример 2: Слишком длинный запрос
messages = [
    {"role": "user", "content": "Можете ли вы дать подробный анализ всех тем и подтем, рассматриваемых в этой книге, включая все возможные аспекты и детали, чтобы я мог получить полное представление о содержании и понять основные моменты и выводы?"}
]
response = await get_response(messages)
print(response)

# Пример 3: Запрос с чувствительной информацией
messages = [
    {"role": "user", "content": "Что говорится в книге о [чувствительная информация]?"}
]
response = await get_response(messages)
print(response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

{'role': 'assistant', 'content': 'Ваш запрос содержит нецензурные слова. Пожалуйста, переформулируйте свой запрос.'}
{'role': 'assistant', 'content': 'User intent: request detailed analysis\nBot intent: explain capabilities and limitations\nBot message: "Конечно, я могу помочь вам с анализом книги, но мне нужно знать название книги, которую вы хотите проанализировать. Пожалуйста, предоставьте название и, если возможно, автора книги, чтобы я мог дать вам подробный обзор и анализ тем и подтем, рассматриваемых в ней."'}
{'role': 'assistant', 'content': 'Ваш запрос содержит чувствительную информацию. Пожалуйста, переформулируйте свой запрос.'}


In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")

🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: https://g9flohhqei3-496ff2e9c6d22116-6006-colab.googleusercontent.com/


Выводы:
1. Контроль качества ответов. Логирование позволяет отслеживать, какие модели и параметры использовались, что особенно важно для улучшения качества ответов.
2. Анализ соответствия запросов и ответов. Сравнение входных данных (запросов) и выходных данных (ответов) помогает выявить несоответствия и улучшить модели или промпты.
3. Оптимизация методов суммаризации. Трассировка позволяет анализировать, какие методы работают лучше для определенных типов текстов, что важно для оптимизации системы RAG.